In [1]:
# --- Instalación de librerías adicionales ---
!pip install polyline

# --- Importaciones ---
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, r2_score
import joblib
import requests
import os
from datetime import datetime
import warnings
import time
import polyline

# --- Importar el gestor de secretos de Kaggle ---
from kaggle_secrets import UserSecretsClient

# --- Configuración Inicial ---
warnings.simplefilter(action='ignore')
pd.set_option('display.max_rows', 100)

print("Librerías importadas y configuración inicial completada.")

Librerías importadas y configuración inicial completada.


In [2]:
# --- Cargar la clave de API desde Kaggle Secrets ---
user_secrets = UserSecretsClient()
API_KEY_GOOGLE = user_secrets.get_secret("GOOGLE_API_KEY")

# --- Variables de Configuración ---
# Ruta representativa en Camp de Túria (Llíria a Bétera)
ORIGIN_COORDS = "39.6253,-0.5961"
DESTINATION_COORDS = "39.5925,-0.4619"

# Rutas de archivos dentro del entorno de Kaggle
KAGGLE_WORKING_DIR = "/kaggle/working/"
HISTORICO_CSV_PATH = os.path.join(KAGGLE_WORKING_DIR, 'historico_trafico_avanzado.csv')
MODELO_PATH = os.path.join(KAGGLE_WORKING_DIR, 'modelo_trafico_avanzado.pkl')

print(f"La API Key ha sido cargada de forma segura.")
print(f"El archivo de datos se guardará en: {HISTORICO_CSV_PATH}")
print(f"El modelo se guardará en: {MODELO_PATH}")

# (Esto va en la Celda 2, después de la definición de MODELO_PATH)

# --- Lista de Rutas en Camp de Túria para la recolección ---
# Puedes añadir, quitar o modificar las rutas que quieras.
LISTA_DE_RUTAS = [
    {
        "nombre": "Llíria -> Bétera",
        "origen": "39.6253,-0.5961",
        "destino": "39.5925,-0.4619"
    },
    {
        "nombre": "Riba-roja -> L'Eliana",
        "origen": "39.5393,-0.5350",
        "destino": "39.5639,-0.5218"
    },
    {
        "nombre": "La Pobla de Vallbona -> Vilamarxant",
        "origen": "39.5919,-0.5486",
        "destino": "39.5694,-0.6214"
    },
    {
        "nombre": "Benissanó -> Nàquera",
        "origen": "39.6105,-0.5701",
        "destino": "39.6200,-0.4133"
    },
    {
        "nombre": "Serra -> Marines",
        "origen": "39.6914,-0.4283",
        "destino": "39.6642,-0.5472"
    }
]

print(f"Se han definido {len(LISTA_DE_RUTAS)} rutas para monitorear en Camp de Túria.")

La API Key ha sido cargada de forma segura.
El archivo de datos se guardará en: /kaggle/working/historico_trafico_avanzado.csv
El modelo se guardará en: /kaggle/working/modelo_trafico_avanzado.pkl
Se han definido 5 rutas para monitorear en Camp de Túria.


In [3]:
# ==============================================================================
# PARTE A: RECOLECTOR DE DATOS AVANZADO (VERSIÓN MULTI-RUTA)
# ==============================================================================

def obtener_datos_de_una_ruta(nombre_ruta, origin_coords, destination_coords):
    """
    Orquesta las llamadas a las APIs de Google para UNA RUTA ESPECÍFICA.
    """
    print(f"--- Recolectando datos para la ruta: {nombre_ruta} ---")
    if API_KEY_GOOGLE == "AQUI_VA_TU_API_KEY" or not API_KEY_GOOGLE:
        print("ADVERTENCIA: No se ha configurado una API Key de Google.")
        return None

    try:
        # 1. Routes API
        routes_data = llamar_routes_api(origin_coords, destination_coords)
        if not routes_data or 'routes' not in routes_data or not routes_data['routes']:
            print(f"Error en Routes API para la ruta {nombre_ruta}.")
            return None
        
        route = routes_data['routes'][0]
        duracion_con_trafico_seg = int(route['duration'].replace('s', ''))
        distancia_metros = route['distanceMeters']
        
        # 2. OMITIDO: Roads API (por el requisito de facturación)
        velocidad_limite_promedio_kmh = 90.0 # Usamos un valor fijo

        # 3. Distance Matrix API
        matrix_data = llamar_distance_matrix_api(origin_coords, destination_coords)
        if not matrix_data:
            print(f"Error en Distance Matrix API para la ruta {nombre_ruta}.")
            return None
        duracion_sin_trafico_seg = matrix_data['duration']['value']

        # Calcular características
        factor_congestion = duracion_con_trafico_seg / duracion_sin_trafico_seg if duracion_sin_trafico_seg > 0 else 1.0

        nuevo_registro = {
            'timestamp': datetime.now(),
            'nombre_ruta': nombre_ruta,  # <-- ¡NUEVA COLUMNA IMPORTANTE!
            'duracion_viaje_seg': duracion_con_trafico_seg,
            'distancia_metros': distancia_metros,
            'velocidad_limite_promedio_kmh': velocidad_limite_promedio_kmh,
            'factor_congestion': round(factor_congestion, 2)
        }
        
        # Guardar en el CSV
        df_nuevo = pd.DataFrame([nuevo_registro])
        df_nuevo.to_csv(HISTORICO_CSV_PATH, mode='a', header=not os.path.exists(HISTORICO_CSV_PATH), index=False)
        print(f"Éxito. Registro para '{nombre_ruta}' guardado.")
        return nuevo_registro

    except requests.exceptions.RequestException as e:
        print(f"Error de red para la ruta {nombre_ruta}: {e}")
        return None
    except Exception as e:
        print(f"Un error inesperado ocurrió en la ruta {nombre_ruta}: {e}")
        return None

# Las funciones de ayuda no necesitan cambios, solo la principal
def llamar_routes_api(origin, destination):
    url = "https://routes.googleapis.com/directions/v2:computeRoutes"
    headers = {'Content-Type': 'application/json','X-Goog-Api-Key': API_KEY_GOOGLE,'X-Goog-FieldMask': 'routes.duration,routes.distanceMeters,routes.polyline'}
    payload = {"origin": {"location": {"latLng": {"latitude": float(origin.split(',')[0]), "longitude": float(origin.split(',')[1])}}},"destination": {"location": {"latLng": {"latitude": float(destination.split(',')[0]), "longitude": float(destination.split(',')[1])}}},"travelMode": "DRIVE","routingPreference": "TRAFFIC_AWARE",}
    response = requests.post(url, json=payload, headers=headers)
    response.raise_for_status()
    return response.json()

def llamar_distance_matrix_api(origin, destination):
    url = (f"https://maps.googleapis.com/maps/api/distancematrix/json?"f"origins={origin}&destinations={destination}&mode=driving&key={API_KEY_GOOGLE}")
    response = requests.get(url)
    response.raise_for_status()
    data = response.json()
    if data['status'] == 'OK' and data['rows'][0]['elements'][0]['status'] == 'OK': return data['rows'][0]['elements'][0]
    else: print(f"Error en Distance Matrix API: {data.get('error_message', data['status'])}"); return None

print("Funciones del recolector de datos (Parte A) adaptadas para multi-ruta.")

Funciones del recolector de datos (Parte A) adaptadas para multi-ruta.


In [4]:
# ==============================================================================
# PARTE B: ENTRENADOR Y PRONOSTICADOR DE ML (VERSIÓN MULTI-RUTA)
# ==============================================================================

def entrenar_y_predecir_multiruta():
    """Carga datos históricos, entrena un modelo y predice el tráfico para múltiples rutas."""
    print("\n--- PARTE B: ENTRENADOR Y PRONOSTICADOR DE ML (MULTI-RUTA) ---")
    
    if not os.path.exists(HISTORICO_CSV_PATH):
        print(f"Error: El archivo '{HISTORICO_CSV_PATH}' no existe. Ejecute el recolector primero.")
        return
        
    df = pd.read_csv(HISTORICO_CSV_PATH, parse_dates=['timestamp'])
    
    if len(df) < 20:
        print(f"ADVERTENCIA: Hay muy pocos datos ({len(df)} registros). La predicción no será precisa.")

    # --- INGENIERÍA DE CARACTERÍSTICAS (MULTI-RUTA) ---
    df['hora'] = df['timestamp'].dt.hour
    df['dia_semana'] = df['timestamp'].dt.dayofweek
    df['es_finde'] = (df['dia_semana'] >= 5).astype(int)
    
    # ¡Importante! Convertimos la columna 'nombre_ruta' en características numéricas
    df_con_rutas = pd.get_dummies(df, columns=['nombre_ruta'], prefix='ruta')
    
    features_base = ['hora', 'dia_semana', 'es_finde', 'distancia_metros', 'velocidad_limite_promedio_kmh']
    # Añadimos las nuevas columnas de ruta (ej: 'ruta_Llíria -> Bétera') a las características
    features_rutas = [col for col in df_con_rutas if col.startswith('ruta_')]
    features_finales = features_base + features_rutas
    
    target = 'duracion_viaje_seg'
    
    X = df_con_rutas[features_finales]
    y = df_con_rutas[target]
    
    if len(df) > 1:
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    else:
        X_train, y_train, X_test, y_test = X, y, pd.DataFrame(), pd.Series()

    # --- ENTRENAR EL MODELO ---
    print("Entrenando el modelo RandomForestRegressor con datos multi-ruta...")
    model = RandomForestRegressor(n_estimators=100, random_state=42, n_jobs=-1)
    model.fit(X_train, y_train)
    joblib.dump(model, MODELO_PATH)
    print(f"Modelo multi-ruta guardado como '{MODELO_PATH}'.\n")

    # --- EVALUAR EL MODELO ---
    if not y_test.empty:
        print("Evaluando el rendimiento del modelo...")
        predictions = model.predict(X_test)
        mae = mean_absolute_error(y_test, predictions)
        r2 = r2_score(y_test, predictions)
        print(f"  - Error Medio Absoluto (MAE): {mae:.2f} segundos")
        print(f"  - Coeficiente R²: {r2:.2f}\n")
    
    # --- PRONOSTICAR PARA TODAS LAS RUTAS ---
    print("Pronosticando las próximas 24 horas para TODAS las rutas conocidas...")
    future_timestamps = pd.date_range(start=datetime.now(), periods=24, freq='h')
    df_pronostico_total = pd.DataFrame()

    # Obtenemos los datos promedio de cada ruta desde el histórico
    datos_por_ruta = df.groupby('nombre_ruta')[['distancia_metros', 'velocidad_limite_promedio_kmh']].mean()

    for nombre_ruta, datos_ruta in datos_por_ruta.iterrows():
        df_futuro = pd.DataFrame({'timestamp': future_timestamps})
        df_futuro['hora'] = df_futuro['timestamp'].dt.hour
        df_futuro['dia_semana'] = df_futuro['timestamp'].dt.dayofweek
        df_futuro['es_finde'] = (df_futuro['dia_semana'] >= 5).astype(int)
        df_futuro['distancia_metros'] = datos_ruta['distancia_metros']
        df_futuro['velocidad_limite_promedio_kmh'] = datos_ruta['velocidad_limite_promedio_kmh']
        
        # Añadimos las columnas de ruta para la predicción
        for ruta_col in features_rutas:
            df_futuro[ruta_col] = 1 if ruta_col == f"ruta_{nombre_ruta}" else 0

        X_futuro = df_futuro[features_finales]
        pronostico = model.predict(X_futuro)
        df_futuro['nombre_ruta'] = nombre_ruta
        df_futuro['duracion_predicha_min'] = (pronostico / 60).round(1)
        df_pronostico_total = pd.concat([df_pronostico_total, df_futuro[['timestamp', 'nombre_ruta', 'duracion_predicha_min']]])
    
    print("\n--- Pronóstico de Duración de Viaje ---")
    print(df_pronostico_total.to_string(index=False))

print("Función del entrenador y pronosticador (Parte B) adaptada para multi-ruta.")

Función del entrenador y pronosticador (Parte B) adaptada para multi-ruta.


In [5]:
import os

# Ruta al archivo que queremos eliminar
ruta_archivo_viejo = '/kaggle/working/historico_trafico_avanzado.csv'

# Comprobamos si el archivo existe y lo borramos
if os.path.exists(ruta_archivo_viejo):
    os.remove(ruta_archivo_viejo)
    print(f"Archivo obsoleto '{ruta_archivo_viejo}' eliminado con éxito.")
else:
    print(f"El archivo obsoleto no existía. ¡Todo listo para empezar!")

El archivo obsoleto no existía. ¡Todo listo para empezar!


In [6]:
# --- INICIO DE LA EJECUCIÓN MULTI-RUTA ---

# 1. Ejecutar el recolector para obtener datos de TODAS las rutas definidas
print("Iniciando recolección de datos para todas las rutas...")
for ruta in LISTA_DE_RUTAS:
    obtener_datos_de_una_ruta(
        nombre_ruta=ruta["nombre"],
        origin_coords=ruta["origen"],
        destination_coords=ruta["destino"]
    )
    # Pequeña pausa para no saturar la API
    time.sleep(1) 
print("\nRecolección de datos completada para esta ejecución.")

# 2. Entrenar con todos los datos disponibles y predecir el futuro para todas las rutas
entrenar_y_predecir_multiruta()

Iniciando recolección de datos para todas las rutas...
--- Recolectando datos para la ruta: Llíria -> Bétera ---
Éxito. Registro para 'Llíria -> Bétera' guardado.
--- Recolectando datos para la ruta: Riba-roja -> L'Eliana ---
Éxito. Registro para 'Riba-roja -> L'Eliana' guardado.
--- Recolectando datos para la ruta: La Pobla de Vallbona -> Vilamarxant ---
Éxito. Registro para 'La Pobla de Vallbona -> Vilamarxant' guardado.
--- Recolectando datos para la ruta: Benissanó -> Nàquera ---
Éxito. Registro para 'Benissanó -> Nàquera' guardado.
--- Recolectando datos para la ruta: Serra -> Marines ---
Éxito. Registro para 'Serra -> Marines' guardado.

Recolección de datos completada para esta ejecución.

--- PARTE B: ENTRENADOR Y PRONOSTICADOR DE ML (MULTI-RUTA) ---
ADVERTENCIA: Hay muy pocos datos (5 registros). La predicción no será precisa.
Entrenando el modelo RandomForestRegressor con datos multi-ruta...
Modelo multi-ruta guardado como '/kaggle/working/modelo_trafico_avanzado.pkl'.

Evalu